## fit force of stripes and monte carlo simulation

In [ ]:
from tensorflow import keras
from tensorflow.keras import optimizers
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline
import random
from neuralgregnet import tools
from neuralgregnet.training import load_model
from tensorflow.keras import backend as K
import scipy.io
import matplotlib.animation as animation
from scipy import interpolate
from matplotlib.pyplot import cm
from scipy import optimize
cmap = plt.get_cmap("Paired")
import commonFunctions as cf

# load data

In [ ]:
genes=['Gt', 'Kni', 'Kr', 'Hb']
cutNaN=35 # to avoid all the nan values
offNaN=965
positions=np.linspace(0,100,1000)
positions=positions[cutNaN:offNaN]
#load data
SortData,sortAge=cf.loadGG(f="gap_data_raw_dorsal_wt.mat",path="DataPetkova/Data/Gap/",
                positions=positions,ageSort=True, age1=40,age2=44,cutPos=False, smooth=False)
#plot
for i,g in enumerate(genes):
    plt.plot(positions[:], SortData[1,i,:].T,color=cmap(i/4), label=g)
plt.xlabel("Positions (% of AP axis)")
plt.ylabel("Concentration")
plt.legend()

#load autoencoder
aeM= load_model("networks/newWT2")
aeM.compile(optimizer=optimizers.Adam(lr=0.01),loss="mse")

# make manifold ( linear or autoencoder)

In [ ]:
cut=cf.find_nearest(positions,35)
off=cf.find_nearest(positions,85)
h1s,h2s,avg1,avg2=cf.makeManifold(SortData,cut=cut,off=off,linear=False,aeM=aeM)

# fit force of stripe on manifold

In [ ]:
# find r and tangent to fit potential directly
#sum_i f'(ri) * r_ij * gradM_j =0 force perpendicular to manifold

petPos=[35.03209091, 43.237355 ,  50.26497085, 55.92945887, 61.85597362 ,67.47041159,75.23405759]
eveIndex=[cf.find_nearest(positions[cut:off],x) for x in petPos]
c=["blue","red","orange","pink","cyan","green","purple"]
#x = np.linspace(-1,1,3) # for linear manifold
x = np.linspace(0,1,2)# for ae manifold
print(x)
plt.figure(figsize=(5,5))
plt.plot(avg1,avg2)
ci=0
for i in eveIndex[:-1]:
    plt.plot(avg1[i],avg2[i],"o", fillstyle='none', markersize=10,color=c[ci])
    ci+=1
plt.plot(avg1[eveIndex[-1]],avg2[eveIndex[-1]],"o", fillstyle='none', markersize=10,color=c[ci] ,label="Eve Stripes")
allDots=[] # calculating all the r_ij * gradM_j
allNorms=[]#calculating the distance between the points
allDots.append(np.zeros(7))
allNorms.append(np.zeros(7))
for stripe in range(1,7):#1,7
    #tangent
    x0 = avg1[eveIndex[stripe]]
    if ( stripe==3 or stripe ==4 or stripe ==5):
        tck = interpolate.splrep((avg1[eveIndex[stripe]-10:eveIndex[stripe]+10])[::-1],(avg2[eveIndex[stripe]-10:eveIndex[stripe]+10])[::-1])
    elif (stripe ==1  or stripe ==6 or stripe==2):
        tck = interpolate.splrep((avg1[eveIndex[stripe]-20:eveIndex[stripe]+20]),(avg2[eveIndex[stripe]-20:eveIndex[stripe]+20]))
    elif stripe==0:
        tck = interpolate.splrep((avg1[eveIndex[stripe]:eveIndex[stripe]+5])[::-1],(avg2[eveIndex[stripe]:eveIndex[stripe]+5])[::-1])

    y0 = interpolate.splev(x0,tck)
    dydx = interpolate.splev(x0,tck,der=1)
    tngnt = lambda x: dydx*x + (y0-dydx*x0)
    
    #normalize vectors
    gradM=[1,tngnt(x)[1]-tngnt(x)[0]]
    gradM=gradM/ np.linalg.norm(gradM)#normalize
    
    #rij
    sumX=0
    sumY=0
    dots=[]
    norms=[]
    for i in range(7):
        if i!=stripe:
            rx=avg1[eveIndex[stripe]]-avg1[eveIndex[i]]
            ry=avg2[eveIndex[stripe]]-avg2[eveIndex[i]]
            ri=[rx,ry]
            norm=np.linalg.norm(ri)
            norms.append(norm)
            ri=ri/norm#normalize
            
            dot=np.dot(ri,gradM)
            dots.append(dot)
            sumX+=rx
            sumY+=ry
        else:
            norms.append(np.nan)
            dots.append(np.nan)
    allDots.append(dots)
    allNorms.append(norms)
    
    
    #plt.plot([0+avg1[eveIndex[stripe]],(sumX+avg1[eveIndex[stripe]])],[0+ avg2[eveIndex[stripe]],(sumY+avg2[eveIndex[stripe]])],'-.', color=c[stripe])
    plt.plot(x,tngnt(x),'--', color=c[stripe])#plot tangent
    #plt.plot([avg1[eveIndex[stripe]+10],avg1[eveIndex[stripe]-10]],[avg2[eveIndex[stripe]+10],avg2[eveIndex[stripe]-10]],':', color=c[stripe])
    plt.legend()
    plt.xlim(0,1)#ae
    plt.ylim(0,1)
    #plt.xlim(-0.75,0.75)#linear
    #plt.ylim(-0.75,0.75)

allDots=np.array(allDots[:])
allNorms=np.array(allNorms[:])
plt.xlabel(r"$h_1$")

In [ ]:
#solve for f
def equations(p):
    params= p
    e=np.zeros((7))
    for j in range(1,7):#1,7
        for i in range(7):
            if i !=j:
                r=allNorms[j][i]
                #try a form for the force
                #a,b,c,d,f=params[0],params[1],params[2],params[3],params[4]
                #e[j]+=((params[2*i +1]) - params[2*i +1]*(allNorms[j][i]**2))*allDots[j][i]
                e[j]+=(((params[i]*np.exp(-params[i]*allNorms[j][i]))*allDots[j][i]))
                       #( params[3*i +1]*(params[3*i]*allNorms[j][i])   +   params[3*i +2]*(params[3*i]*allNorms[j][i])**2))*allDots[j][i])
                #e[j]+=(4*params[2*i]*( 12*(params[2*i +1]**12)/(allNorms[j][i])**13 - 
                 #   6*(params[2*i +1]**6)/(allNorms[j][i])**7)*allDots[j][i])
                #e[j]+=(-a*np.exp(-b*r) *(d + 2 *f *r - b *(c + r *(d + f* r))))*allDots[j][i]
                #e[j]+=(a*np.exp(-b*r)*(c + d*r + f*r**2))*allDots[j][i]
                #e[j]+=params[i]*(allNorms[j][i]**(-2))*allDots[j][i]
                #e[j]+=params[i]*(allNorms[j][i]**(-3))*allDots[j][i]

    return (e[1], e[2],e[3],e[4],e[5],e[6])#,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)

p=  scipy.optimize.least_squares(equations, x0=[40,40,40,40,40,40,40], bounds=(0,np.inf))#,tr_solver= 'lsmr')#, loss='huber')#,1,1,1,1,1,1,1,1,1,1,1,1,1,1))

print( equations(p['x']))
p['x']

In [ ]:
#plot force and potential
a,b= 95.22079372,95.22079372
r=np.linspace(0,1,100)
f=a*np.exp(-b*r)
#f=4*a*( 12*(b**12)/(r)**13 - 6*(b**6)/(r)**7)
#f=a/r**2
#f=a - b*r**2
pot=a*np.exp(-b*r)/b
#p=4*a*( (b**12)/(r)**12 - (b**6)/(r)**6)
#p=a/r
#p= -(a*r -b*(r**3)/3)
plt.figure()
plt.plot(r,f)
plt.figure()
plt.plot(r,pot)
plt.xlabel("r")
plt.ylabel("p")

# Monte carlo simulation

In [ ]:
import random
#monte carlo
beta=10
changeRate=0.0001
#potential
def P2(x,y,x1,y1,s1,s2):
    l=p['x']
    r=((x-x1)**2 + (y-y1)**2)**(0.5)
    #return (l[s1]*np.exp(-l[7]*r)) +  ( l[s2]*np.exp(-l[7]*r))
    #return (2*a *np.exp(-b* r) *(2 *f + b *(d + 2* f *r) + (b**2) *(c + r* (d + f *r))))/b**3
    #return (l[s1*2] * np.exp(-l[s1*2 +1]*r)/l[s1*2 +1]) +  (l[s2*2] * np.exp(-l[s2*2 +1]*r)/l[s2*2 +1])
    return ( np.exp(-l[s1]*r) +  np.exp(-l[s2]*r))
    #return  4*l[s1*2]*((l[s1*2 +1]/r)**12 - (l[s1*2 +1]/r)**6) + 4*l[s2*2]*((l[s2*2 +1]/r)**12 - (l[s2*2 +1]/r)**6)
    #return -(l[2*s1]*(r)- l[2*s1 +1]*(r**3)/3.0) - (l[2*s2]*(r) - l[2*s2 +1]*(r**3)/3.0)
    #return  2*94.99999116 *np.exp(- 102.00385341*r)/102.00385341
    #return l[s1]/(r)  + l[s2]/(r)

#stable temperature
#place points randomly on manifold
la1=len(avg1)
index=[random.randint(0,la1-1) for x in range(7)]
index=sorted(index)
time=10000

allIndex=np.zeros((time,7), dtype=int)
#run monte carlo for many iterations
for t in range(time):  #move by 5
    #pick one of the seven points at random 
    point=random.randint(0,6)#make sure this is from random not np
    #energy for that point
    pointEnergy=0
    for j in range(7):
        if j!=point:
            pointEnergy+= P2(avg1[index[point]],avg2[index[point]],avg1[index[j]],avg2[index[j]],j,point)
    #pick alpha btw 0,1
    alpha=random.uniform(0,1)
    
    #tentative new position
    tryIndex= (index[point]+(random.choice((-1,1))*5))%la1
    while (tryIndex == index[0] or tryIndex == index[1] or tryIndex == index[2]or 
           tryIndex == index[3] or tryIndex == index[4] or tryIndex == index[5]  or
           tryIndex == index[6]):# or tryIndex>=la1 or tryIndex<0):
            tryIndex = (index[point]+(random.choice((-1,1))*10)) % la1
    #energy for tentative point
    tryPointEnergy=0
    for j in range(7):
        if j!=point:
            tryPointEnergy+= P2(avg1[tryIndex],avg2[tryIndex],avg1[index[j]],avg2[index[j]],j,point) 
    deltaE= tryPointEnergy-pointEnergy
    if alpha <min(1,np.exp(-beta*deltaE)):
        index[point]=tryIndex
        index=sorted(index)
    allIndex[t]=np.array(index)
firstIndex=allIndex.copy()    
    
    

#continue but lower T increase beta
index=firstIndex[-1].copy()
time=155000
allIndex=np.zeros((time,7), dtype=int)
#run monte carlo for many iterations
for t in range(int(time/2)):  #move by 5
    beta=beta+(changeRate*t)
    #pick one of the seven points at random 
    point=random.randint(0,6)
    #energy for that point
    pointEnergy=0
    for j in range(7):
        if j!=point:
            pointEnergy+= P2(avg1[index[point]],avg2[index[point]],avg1[index[j]],avg2[index[j]],j,point)
    #pick alpha btw 0,1
    alpha=random.uniform(0,1)
    
    #tentative new position
    tryIndex= (index[point]+(random.choice((-1,1))*5))%la1
    while (tryIndex == index[0] or tryIndex == index[1] or tryIndex == index[2]or 
           tryIndex == index[3] or tryIndex == index[4] or tryIndex == index[5]  or
           tryIndex == index[6]):# or tryIndex>=la1 or tryIndex<0):
            tryIndex = (index[point]+(random.choice((-1,1))*10)) % la1
    #energy for tentative point
    tryPointEnergy=0
    for j in range(7):
        if j!=point:
            tryPointEnergy+= P2(avg1[tryIndex],avg2[tryIndex],avg1[index[j]],avg2[index[j]],j,point) 
    deltaE= tryPointEnergy-pointEnergy
    if alpha <min(1,np.exp(-beta*deltaE)):
        index[point]=tryIndex
        index=sorted(index)
    allIndex[t]=np.array(index)
    
for t in range(int(time/2), time): #move by 1
    beta=beta+(changeRate*t)
    #pick one of the seven points at random 
    point=random.randint(0,6)
    #energy for that point
    pointEnergy=0
    for j in range(7):
        if j!=point:
            pointEnergy+= P2(avg1[index[point]],avg2[index[point]],avg1[index[j]],avg2[index[j]],j,point)
    #pick alpha btw 0,1
    alpha=random.uniform(0,1)
    
    #tentative new position
    tryIndex= (index[point]+(random.choice((-1,1))*1))%la1
    while (tryIndex == index[0] or tryIndex == index[1] or tryIndex == index[2]or 
           tryIndex == index[3] or tryIndex == index[4] or tryIndex == index[5]  or
           tryIndex == index[6]):# or tryIndex>=la1 or tryIndex<0):
            tryIndex = (index[point]+(random.choice((-1,1))*5)) % la1
    #energy for tentative point
    tryPointEnergy=0
    for j in range(7):
        if j!=point:
            tryPointEnergy+= P2(avg1[tryIndex],avg2[tryIndex],avg1[index[j]],avg2[index[j]],j,point) 
    deltaE= tryPointEnergy-pointEnergy
    if alpha <min(1,np.exp(-beta*deltaE)):
        index[point]=tryIndex
        index=sorted(index)
    allIndex[t]=np.array(index)

lastIndex=allIndex.copy()
totalIndex=np.concatenate((firstIndex,lastIndex))
#index=sorted(index)

In [ ]:
#plot result of monte carlo
plt.figure(figsize=(5,5))
plt.xlim(-0.75,0.75)
plt.ylim(-0.75,0.75)
plt.xlim(0,1)
plt.ylim(0,1)
plt.plot(avg1,avg2)
petPos=[35.03209091, 43.237355 ,  50.26497085, 55.92945887, 61.85597362 ,67.47041159,75.23405759]
eveIndex=[cf.find_nearest(positions[cut:off],x) for x in petPos]
c=["blue","red","orange","pink","cyan","green","purple"]
ci=0
for i in eveIndex[:-1]:
    plt.plot(avg1[i],avg2[i],"o", fillstyle='none', markersize=10,color=c[ci])
    ci+=1
plt.plot(avg1[eveIndex[-1]],avg2[eveIndex[-1]],"o", fillstyle='none', markersize=10,color=c[ci] ,label="Petkova")

ci=0
for i in index[:-1]:
    plt.plot(avg1[i],avg2[i],".",  markersize=10,color=c[ci])
    ci+=1
plt.plot(avg1[index[-1]],avg2[index[-1]],".",  markersize=10,color=c[ci], label="Monte Carlo")
print("last index: " +str(index))
plt.legend()
plt.xlabel(r"$h_1$")
plt.ylabel(r"$h_2$")

In [ ]:
#movie of monte carlo


fig = plt.figure(figsize=(5,5))
#ax = plt.axes(xlim=(-0.75, 0.75), ylim=(-0.75, 0.75))
ax = plt.axes(xlim=(0, 1), ylim=(0, 1))
plt.plot(avg1,avg2)
line, = ax.plot([], [], lw=2)
time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)
lines = []

ci=0
for i in eveIndex[:-1]:
    plt.plot(avg1[i],avg2[i],"o", fillstyle='none', markersize=10,color=c[ci])
    ci+=1
plt.plot(avg1[eveIndex[-1]],avg2[eveIndex[-1]],"o", fillstyle='none', markersize=10,color=c[ci] ,label="Petkova2")

for index in range(7):
    lobj = ax.plot([],[],'.',lw=2)[0]
    lines.append(lobj)
def init():
    for line in lines:
        line.set_data([],[])
    time_text.set_text('')
    return lines


def animate(i):
    time_text.set_text('t = %.1f' %(i*300))
    for lnum,line in enumerate(lines):
        line.set_data(avg1[totalIndex[::300][i][lnum]], avg2[totalIndex[::300][i][lnum]])

    return lines

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=totalIndex[::300].shape[0], interval=200, blit=True)

plt.xlabel(r"$h_1$")
plt.ylabel(r"$h_2$")
#plt.title("Manifold for wild type (mutant data) in time")
#plt.show()
Writer = animation.writers['ffmpeg']
writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)
anim.save('movies/potTotb.mp4', writer=writer)

In [ ]:
#many monte carlo to have a distribution of final state
import random
distIndex=[]

    #monte carlo
beta=1000000000
changeRate=100

def P2(x,y,x1,y1,s1,s2):
    l=[40.09263607, 40.00000332, 86.28460419, 86.28461208, 40.00001279,
       83.56344578, 83.56245758]
    r=((x-x1)**2 + (y-y1)**2)**(0.5)
    return ( np.exp(-l[s1]*r) +  np.exp(-l[s2]*r))

for num in range(38):# 38 different monte carlo
    print(num)

    #stable temperature
    #place points randomly on manifold
    la1=len(avg1)
    index=[random.randint(0,la1-1) for x in range(7)]
    index=sorted(index)
    time=10000

    allIndex=np.zeros((time,7), dtype=int)
    #run monte carlo for many iterations
    for t in range(time):
        #pick one of the seven points at random 
        point=random.randint(0,6)
        #energy for that point
        pointEnergy=0
        for j in range(7):
            if j!=point:
                pointEnergy+= P2(avg1[index[point]],avg2[index[point]],avg1[index[j]],avg2[index[j]],j,point)
        #pick alpha btw 0,1
        alpha=random.uniform(0,1)

        #tentative new position
        tryIndex= (index[point]+(random.choice((-1,1))*5))%la1
        while (tryIndex == index[0] or tryIndex == index[1] or tryIndex == index[2]or 
               tryIndex == index[3] or tryIndex == index[4] or tryIndex == index[5]  or
               tryIndex == index[6]):# or tryIndex>=la1 or tryIndex<0):
                tryIndex = (index[point]+(random.choice((-1,1))*10)) % la1
        #energy for tentative point
        tryPointEnergy=0
        for j in range(7):
            if j!=point:
                tryPointEnergy+= P2(avg1[tryIndex],avg2[tryIndex],avg1[index[j]],avg2[index[j]],j,point) 
        deltaE= tryPointEnergy-pointEnergy
        if alpha <min(1,np.exp(-beta*deltaE)):
            index[point]=tryIndex
            index=sorted(index)
        allIndex[t]=np.array(index)
    firstIndex=allIndex.copy()    



    #continue but lower T increase beta
    index=firstIndex[-1].copy()
    time=155000
    allIndex=np.zeros((time,7), dtype=int)
    #run monte carlo for many iterations
    for t in range(int(time/2)):
        beta=beta+(changeRate*t)
        #pick one of the seven points at random 
        point=random.randint(0,6)
        #energy for that point
        pointEnergy=0
        for j in range(7):
            if j!=point:
                pointEnergy+= P2(avg1[index[point]],avg2[index[point]],avg1[index[j]],avg2[index[j]],j,point)
        #pick alpha btw 0,1
        alpha=random.uniform(0,1)

        #tentative new position
        tryIndex= (index[point]+(random.choice((-1,1))*5))%la1
        while (tryIndex == index[0] or tryIndex == index[1] or tryIndex == index[2]or 
               tryIndex == index[3] or tryIndex == index[4] or tryIndex == index[5]  or
               tryIndex == index[6]):# or tryIndex>=la1 or tryIndex<0):
                tryIndex = (index[point]+(random.choice((-1,1))*10)) % la1
        #energy for tentative point
        tryPointEnergy=0
        for j in range(7):
            if j!=point:
                tryPointEnergy+= P2(avg1[tryIndex],avg2[tryIndex],avg1[index[j]],avg2[index[j]],j,point) 
        deltaE= tryPointEnergy-pointEnergy
        if alpha <min(1,np.exp(-beta*deltaE)):
            index[point]=tryIndex
            index=sorted(index)
        allIndex[t]=np.array(index)

    for t in range(int(time/2), time):
        beta=beta+(changeRate*t)
        #pick one of the seven points at random 
        point=random.randint(0,6)
        #energy for that point
        pointEnergy=0
        for j in range(7):
            if j!=point:
                pointEnergy+= P2(avg1[index[point]],avg2[index[point]],avg1[index[j]],avg2[index[j]],j,point)
        #pick alpha btw 0,1
        alpha=random.uniform(0,1)

        #tentative new position
        tryIndex= (index[point]+(random.choice((-1,1))*1))%la1
        while (tryIndex == index[0] or tryIndex == index[1] or tryIndex == index[2]or 
               tryIndex == index[3] or tryIndex == index[4] or tryIndex == index[5]  or
               tryIndex == index[6]):# or tryIndex>=la1 or tryIndex<0):
                tryIndex = (index[point]+(random.choice((-1,1))*5)) % la1
        #energy for tentative point
        tryPointEnergy=0
        for j in range(7):
            if j!=point:
                tryPointEnergy+= P2(avg1[tryIndex],avg2[tryIndex],avg1[index[j]],avg2[index[j]],j,point) 
        deltaE= tryPointEnergy-pointEnergy
        if alpha <min(1,np.exp(-beta*deltaE)):
            index[point]=tryIndex
            index=sorted(index)
        allIndex[t]=np.array(index)

    lastIndex=allIndex.copy()
    totalIndex=np.concatenate((firstIndex,lastIndex))
    #index=sorted(index)
    distIndex.append(index)

In [ ]:
#plot distribution for final state that have an okay position for 7th eve stripe
di=np.array(distIndex)
#"""
i=0
while i < (di.shape[0]):
    print(i,di.shape[0])
    if di[i][6] >419: # for every simulation that failed and put 7 too far
        print(di[i][1])
        di=np.delete(di,(i),axis=0)
    else:
        i+=1
#"""        
print(di.shape)

for j in range(7):
    diP=[positions[cut+x] for x in di[:,j]]
    plt.hist(diP,bins=len(diP), color=c[j])
#plt.hist(di[:,1],bins=100)
#plt.axvline(positions[cut+419])
#plt.axvline(positions[cut+14])

# Mutant prediction
doesn't really work

In [ ]:
#load data
files=["gap_data_raw_dorsal_wt_osk.mat","gap_data_raw_dorsal_wt_etsl.mat","gap_data_raw_dorsal_wt_bcdE1.mat","gap_data_raw_dorsal_wt_bcd_tsl.mat",
      "gap_data_raw_dorsal_wt_bcd_osk.mat","gap_data_raw_dorsal_wt_bcd_only_germline_clones.mat","gap_data_raw_dorsal_wt_bcd_nos_tsl.mat"]
name=['osk','etsl','bcdE1','bcd-tsl','bcd-osk','bcdGermline','bcd-nos-tsl']
m=4
print(name[m])
mutantData,sortAgeM=cf.loadGG(f=files[m],path="DataPetkova/Data/Gap/",
                positions=positions,ageSort=True, age1=40,age2=44,cutPos=False, smooth=False, genotype=2)
#plot
for i,g in enumerate(genes):
    plt.plot(positions[:], mutantData[1,i,:].T,color=cmap(i/4), label=g)
plt.xlabel("Positions (% of AP axis)")
plt.ylabel("Concentration")
plt.legend()

In [ ]:
cut=cf.find_nearest(positions,35)
off=cf.find_nearest(positions,85)
h1s,h2s,avg1,avg2=cf.makeManifold(mutantData,cut=cut,off=off,linear=True,aeM=aeM)

In [ ]:
import random
#monte carlo
beta=100000000
changeRate=1000
nbPeaks=5

def P2(x,y,x1,y1,s1,s2):
    #l=[40.09263607, 40.00000332, 86.28460419, 86.28461208, 40.00001279, 83.56344578, 83.56245758]
    l=[83.56245758, 86.28461208, 40.00001279, 83.56344578, 83.56245758]
    r=((x-x1)**2 + (y-y1)**2)**(0.5)
    #return (l[s1]*np.exp(-l[7]*r)) +  ( l[s2]*np.exp(-l[7]*r))
    #return (2*a *np.exp(-b* r) *(2 *f + b *(d + 2* f *r) + (b**2) *(c + r* (d + f *r))))/b**3
    #return (l[s1*2] * np.exp(-l[s1*2 +1]*r)/l[s1*2 +1]) +  (l[s2*2] * np.exp(-l[s2*2 +1]*r)/l[s2*2 +1])
    return ( np.exp(-l[s1]*r) +  np.exp(-l[s2]*r))
    #return  4*l[s1*2]*((l[s1*2 +1]/r)**12 - (l[s1*2 +1]/r)**6) + 4*l[s2*2]*((l[s2*2 +1]/r)**12 - (l[s2*2 +1]/r)**6)
    #return -(l[2*s1]*(r)- l[2*s1 +1]*(r**3)/3.0) - (l[2*s2]*(r) - l[2*s2 +1]*(r**3)/3.0)
    #return  2*94.99999116 *np.exp(- 102.00385341*r)/102.00385341
    #return l[s1]/(r)  + l[s2]/(r)

#stable temperature
#place points randomly on manifold
la1=len(avg1)
index=[random.randint(0,la1-1) for x in range(nbPeaks)]
index=sorted(index)
time=10000

allIndex=np.zeros((time,nbPeaks), dtype=int)
#run monte carlo for many iterations
for t in range(time):
    #pick one of the seven points at random 
    point=random.randint(0,nbPeaks-1)#make sure this is from random not np
    #energy for that point
    pointEnergy=0
    for j in range(nbPeaks):
        if j!=point:
            pointEnergy+= P2(avg1[index[point]],avg2[index[point]],avg1[index[j]],avg2[index[j]],j,point)
    #pick alpha btw 0,1
    alpha=random.uniform(0,1)
    
    #tentative new position
    tryIndex= (index[point]+(random.choice((-1,1))*5))%la1
    while (tryIndex == index[0] or tryIndex == index[1] or tryIndex == index[2]or 
           tryIndex == index[3]):# or tryIndex == index[4] or tryIndex == index[5]  or
           #tryIndex == index[6]):# or tryIndex>=la1 or tryIndex<0):
            tryIndex = (index[point]+(random.choice((-1,1))*10)) % la1
    #energy for tentative point
    tryPointEnergy=0
    for j in range(nbPeaks):
        if j!=point:
            tryPointEnergy+= P2(avg1[tryIndex],avg2[tryIndex],avg1[index[j]],avg2[index[j]],j,point) 
    deltaE= tryPointEnergy-pointEnergy
    if alpha <min(1,np.exp(-beta*deltaE)):
        index[point]=tryIndex
        index=sorted(index)
    allIndex[t]=np.array(index)
firstIndex=allIndex.copy()    
    
    

#continue but lower T increase beta
index=firstIndex[-1].copy()
time=155000
allIndex=np.zeros((time,nbPeaks), dtype=int)
#run monte carlo for many iterations
for t in range(int(time/2)):
    #beta=beta+(5*t)
    beta=beta+(changeRate*t)
    #pick one of the seven points at random 
    point=random.randint(0,nbPeaks-1)
    #energy for that point
    pointEnergy=0
    for j in range(nbPeaks):
        if j!=point:
            pointEnergy+= P2(avg1[index[point]],avg2[index[point]],avg1[index[j]],avg2[index[j]],j,point)
    #pick alpha btw 0,1
    alpha=random.uniform(0,1)
    
    #tentative new position
    tryIndex= (index[point]+(random.choice((-1,1))*5))%la1
    while (tryIndex == index[0] or tryIndex == index[1] or tryIndex == index[2]or 
           tryIndex == index[3] ):#or tryIndex == index[4] or tryIndex == index[5]  or
           #tryIndex == index[6]):# or tryIndex>=la1 or tryIndex<0):
            tryIndex = (index[point]+(random.choice((-1,1))*10)) % la1
    #energy for tentative point
    tryPointEnergy=0
    for j in range(nbPeaks):
        if j!=point:
            tryPointEnergy+= P2(avg1[tryIndex],avg2[tryIndex],avg1[index[j]],avg2[index[j]],j,point) 
    deltaE= tryPointEnergy-pointEnergy
    if alpha <min(1,np.exp(-beta*deltaE)):
        index[point]=tryIndex
        index=sorted(index)
    allIndex[t]=np.array(index)
    
for t in range(int(time/2), time):
    #beta=beta+(5*t)
    beta=beta+(changeRate*t)
    #pick one of the seven points at random 
    point=random.randint(0,nbPeaks-1)
    #energy for that point
    pointEnergy=0
    for j in range(nbPeaks):
        if j!=point:
            pointEnergy+= P2(avg1[index[point]],avg2[index[point]],avg1[index[j]],avg2[index[j]],j,point)
    #pick alpha btw 0,1
    alpha=random.uniform(0,1)
    
    #tentative new position
    tryIndex= (index[point]+(random.choice((-1,1))*1))%la1
    while (tryIndex == index[0] or tryIndex == index[1] or tryIndex == index[2]or 
           tryIndex == index[3]):# or tryIndex == index[4] or tryIndex == index[5]  or
           #tryIndex == index[6]):# or tryIndex>=la1 or tryIndex<0):
            tryIndex = (index[point]+(random.choice((-1,1))*5)) % la1
    #energy for tentative point
    tryPointEnergy=0
    for j in range(nbPeaks):
        if j!=point:
            tryPointEnergy+= P2(avg1[tryIndex],avg2[tryIndex],avg1[index[j]],avg2[index[j]],j,point) 
    deltaE= tryPointEnergy-pointEnergy
    if alpha <min(1,np.exp(-beta*deltaE)):
        index[point]=tryIndex
        index=sorted(index)
    allIndex[t]=np.array(index)

lastIndex=allIndex.copy()
totalIndex=np.concatenate((firstIndex,lastIndex))
#index=sorted(index)

In [ ]:
%matplotlib inline
plt.figure(figsize=(5,5))
plt.xlim(-0.75,0.75)
plt.ylim(-0.75,0.75)
plt.plot(avg1,avg2)
petPos=[23.35669002, 39.40607274, 48.48181515, 55.95595596 ,69.58625292]
eveIndex=[cf.find_nearest(positions[cut:off],x) for x in petPos]
c=["blue","red","orange","pink","cyan","green","purple"]
ci=0
for i in eveIndex[:-1]:
    plt.plot(avg1[i],avg2[i],"o", fillstyle='none', markersize=10,color=c[ci])
    ci+=1
plt.plot(avg1[eveIndex[-1]],avg2[eveIndex[-1]],"o", fillstyle='none', markersize=10,color=c[ci] ,label="Petkova")

ci=0
for i in index[:-1]:
    plt.plot(avg1[i],avg2[i],".",  markersize=10,color=c[ci])
    ci+=1
plt.plot(avg1[index[-1]],avg2[index[-1]],".",  markersize=10,color=c[ci], label="Monte Carlo")
print("last index: " +str(index))
plt.legend()
plt.xlabel(r"$h_1$")
plt.ylabel(r"$h_2$")

In [ ]:
#movie of manifold
#%matplotlib inline


#%matplotlib qt
#%matplotlib notebook


fig = plt.figure(figsize=(5,5))
ax = plt.axes(xlim=(-0.75, 0.75), ylim=(-0.75, 0.75))
plt.plot(avg1,avg2)
line, = ax.plot([], [], lw=2)
time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)
lines = []

ci=0
for i in eveIndex[:-1]:
    plt.plot(avg1[i],avg2[i],"o", fillstyle='none', markersize=10,color=c[ci])
    ci+=1
plt.plot(avg1[eveIndex[-1]],avg2[eveIndex[-1]],"o", fillstyle='none', markersize=10,color=c[ci] ,label="Petkova2")

for index in range(nbPeaks):
    lobj = ax.plot([],[],'.',lw=2)[0]
    lines.append(lobj)
def init():
    for line in lines:
        line.set_data([],[])
    time_text.set_text('')
    return lines


def animate(i):
    time_text.set_text('t = %.1f' %(i*300))
    for lnum,line in enumerate(lines):
        line.set_data(avg1[totalIndex[::300][i][lnum]], avg2[totalIndex[::300][i][lnum]])

    return lines

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=totalIndex[::300].shape[0], interval=200, blit=True)

plt.xlabel(r"$h_1$")
plt.ylabel(r"$h_2$")
#plt.title("Manifold for wild type (mutant data) in time")
#plt.show()
Writer = animation.writers['ffmpeg']
writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)
anim.save('movies/potTotb.mp4', writer=writer)